#Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pylab import rcParams
#Checking for stationarity
# Importing the necessary package
from statsmodels.tsa.stattools import adfuller

#GRaphing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

# sharper plots
%config InlineBackend.figure_format = 'retina'
%matplotlib inline



In [ ]:
from fbprophet import Prophet

import logging
logging.getLogger().setLevel(logging.ERROR)


In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

# Initialize plotly
init_notebook_mode(connected=True)

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook



%matplotlib inline

#Functions Being implemented

In [ ]:
#shows how accurate our transfomred datasets are compared to thier actual forecasts 
def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Lower Bound', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"))
    upper_bound = create_go('Upper Bound', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Forecast', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,
                       marker=dict(color="red"))
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='Posts'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [ ]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model. 
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
#The inverse difference function


# from pmdarima.utils import diff_inv

# def inv_diff (df_orig_column,df_diff_column, periods):
#     # Generate np.array for the diff_inv function - it includes first n values(n = 
#     # periods) of original data & further diff values of given periods
#     value = np.array(df_orig_column[:periods].tolist()+df_diff_column[periods:].tolist())

#     # Generate np.array with inverse diff
#     inv_diff_vals = diff_inv(value, periods,1 )[periods:]
#     return inv_diff_vals


In [ ]:
#Reversing the transformations
# https://stackoverflow.com/questions/59922291/how-to-reverse-a-seasonal-log-difference-of-timeseries-in-python
#https://stackoverflow.com/questions/48428679/opposite-of-df-diff-in-pandas
#https://stackoverflow.com/questions/34918013/undo-a-series-diff


#Reading in the Data and Partitioning Trend lines By Category

In [ ]:
rcParams['figure.figsize'] = 10, 6


In [ ]:
mesa_az = pd.read_csv('Mesa_Arizona_Month_by_month.csv') #contains dates until December 2021
mesa_az.head()

,Report Month Date,Report Month Year,Report Month Name,Report Year Month,Tax Type,Business Class,Business Class Description,Category,Category Code Description,Amount,Comparable Amount
0,07/01/2015 12:00:00 AM,2015,Jul,201507,TLT,NaN,NaN,TLT,TLT,159234.43,159234.43
1,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,NaN,NaN,10,Restaurants and Bars,1004732.95,1004732.95
2,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,NaN,NaN,11,Amusements,127152.32,127152.32
3,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,NaN,NaN,12,Rentals,1913641.89,1913641.91
4,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,NaN,NaN,15,Miscellaneous,38591.32,38591.32


In [ ]:
mesa_az = mesa_az.drop(['Business Class', 'Business Class Description'], axis = 1)
mesa_az.head()

,Report Month Date,Report Month Year,Report Month Name,Report Year Month,Tax Type,Category,Category Code Description,Amount,Comparable Amount
0,07/01/2015 12:00:00 AM,2015,Jul,201507,TLT,TLT,TLT,159234.43,159234.43
1,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,10,Restaurants and Bars,1004732.95,1004732.95
2,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,11,Amusements,127152.32,127152.32
3,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,12,Rentals,1913641.89,1913641.91
4,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,15,Miscellaneous,38591.32,38591.32


In [ ]:
mesa_az.isnull().sum()

Report Month Date            0
Report Month Year            0
Report Month Name            0
Report Year Month            0
Tax Type                     0
Category                     0
Category Code Description    0
Amount                       0
Comparable Amount            0
dtype: int64

In [ ]:
#Extracting the month number
mesa_az['Month Number'] = mesa_az['Report Year Month'].astype(str).str[4:]

In [ ]:
#Extracting the Year-Month
mesa_az['Year-Month'] = mesa_az['Report Month Year'].astype(str) + '-' + mesa_az['Month Number'].astype(str)
mesa_az.head(2)

,Report Month Date,Report Month Year,Report Month Name,Report Year Month,Tax Type,Category,Category Code Description,Amount,Comparable Amount,Month Number,Year-Month
0,07/01/2015 12:00:00 AM,2015,Jul,201507,TLT,TLT,TLT,159234.43,159234.43,07,2015-07
1,07/01/2015 12:00:00 AM,2015,Jul,201507,TPT,10,Restaurants and Bars,1004732.95,1004732.95,07,2015-07


In [ ]:
#groupby (['Year-Month', 'Category'])['Amount'].sum()
total_amount_cat = mesa_az.groupby(['Category Code Description', 'Year-Month'], as_index = False)['Amount'].sum()
total_amount_cat

,Category Code Description,Year-Month,Amount
0,Amusements,2015-07,127152.32
1,Amusements,2015-08,96022.36
2,Amusements,2015-09,84270.61
3,Amusements,2015-10,92539.64
4,Amusements,2015-11,133316.53
...,...,...,...
764,Utilities,2021-05,1195882.73
765,Utilities,2021-06,1774064.47
766,Utilities,2021-07,1855668.51
767,Utilities,2021-08,1987163.27


In [ ]:
meza_total_monthly = mesa_az.groupby('Year-Month', as_index = False)['Amount'].sum()
meza_total_monthly.head()

,Year-Month,Amount
0,2015-07,11938700.80
1,2015-08,11974648.06
2,2015-09,11986411.59
3,2015-10,12225716.45
4,2015-11,12408690.93


In [ ]:
meza_total_monthly.tail()

,Year-Month,Amount
70,2021-05,22605975.25
71,2021-06,23661927.82
72,2021-07,22512287.08
73,2021-08,22845875.08
74,2021-09,23666698.18


In [ ]:
px.line(total_amount_cat, 
        x="Year-Month", 
        y="Amount", 
        color="Category Code Description", 
        #text="Revenue",
       title="Sales Tax Revenue by Category")

In [ ]:
total_amount_cat['Category Code Description'].unique()

array(['Amusements', 'Communications', 'Contracting', 'Miscellaneous',
       'Printing and Advertising', 'Publishing', 'Rentals',
       'Restaurants and Bars', 'Retail Sales', 'TLT', 'Utilities'],
      dtype=object)

In [ ]:
#These are the categories that had trend lines that were not stationary - we wo
total_amount_cat_amusement = total_amount_cat[total_amount_cat['Category Code Description'] == 'Amusements']
# total_amount_cat_amusement = total_amount_cat_amusement[:-3]

total_amount_cat_cont = total_amount_cat[total_amount_cat['Category Code Description'] == 'Contracting']
# total_amount_cat_cont = total_amount_cat_cont[:-3]

total_amount_cat_rent = total_amount_cat[total_amount_cat['Category Code Description'] == 'Rentals']
# total_amount_cat_rent = total_amount_cat_rent[:-3]

total_amount_cat_rest_n_bars = total_amount_cat[total_amount_cat['Category Code Description'] == 'Restaurants and Bars']
# total_amount_cat_rest_n_bars = total_amount_cat_rest_n_bars[:-3]


total_amount_cat_retails = total_amount_cat[total_amount_cat['Category Code Description'] == 'Retail Sales']
# total_amount_cat_retails = total_amount_cat_retails[:-3]

total_amount_cat_tlt = total_amount_cat[total_amount_cat['Category Code Description'] == 'TLT']
# total_amount_cat_tlt = total_amount_cat_tlt[:-3]

total_amount_cat_util = total_amount_cat[total_amount_cat['Category Code Description'] == 'Utilities']
# total_amount_cat_util = total_amount_cat_util[:-3]

In [ ]:
#these are tje categories that had trend lines that seemed to be more stationary -> we will treat thm as one later
total_amount_cat_publ = total_amount_cat[total_amount_cat['Category Code Description'] == 'Publishing']
# total_amount_cat_publ =total_amount_cat_publ[:-3]

total_amount_cat_misc = total_amount_cat[total_amount_cat['Category Code Description'] == 'Miscellaneous']
# total_amount_cat_misc = total_amount_cat_misc[:-3]

total_amount_cat_comm = total_amount_cat[total_amount_cat['Category Code Description'] == 'Communications']
# total_amount_cat_comm = total_amount_cat_comm[:-3]

total_amount_cat_PandA = total_amount_cat[total_amount_cat['Category Code Description'] == 'Printing and Advertising']
# total_amount_cat_PandA = total_amount_cat_PandA[:-3]

In [ ]:
#Dropppin the Category Code Description column for all of these
total_amount_cat_amusement = total_amount_cat_amusement.drop('Category Code Description', axis = 1)
total_amount_cat_comm = total_amount_cat_comm.drop('Category Code Description', axis = 1)
total_amount_cat_cont = total_amount_cat_cont.drop('Category Code Description', axis = 1)
total_amount_cat_misc = total_amount_cat_misc.drop('Category Code Description', axis = 1)
total_amount_cat_PandA = total_amount_cat_PandA.drop('Category Code Description', axis = 1)
total_amount_cat_publ = total_amount_cat_publ.drop('Category Code Description', axis = 1)
total_amount_cat_rent = total_amount_cat_rent.drop('Category Code Description', axis = 1)
total_amount_cat_rest_n_bars = total_amount_cat_rest_n_bars.drop('Category Code Description', axis = 1)
total_amount_cat_retails = total_amount_cat_retails.drop('Category Code Description', axis = 1)
total_amount_cat_tlt = total_amount_cat_tlt.drop('Category Code Description', axis = 1)
total_amount_cat_util = total_amount_cat_util.drop('Category Code Description', axis = 1)

In [ ]:
#finding the aggregated trend
#combining TLT, printing and Advertisement, Publishing, amusement

tlt_pandA_publ_amuesement_dfs = [total_amount_cat_tlt, total_amount_cat_PandA , total_amount_cat_publ, total_amount_cat_amusement]
tlt_pandA_publ_amuesement_df = pd.concat(tlt_pandA_publ_amuesement_dfs)
tlt_pandA_publ_amuesement_df_monthly = tlt_pandA_publ_amuesement_df.groupby('Year-Month', as_index = False)['Amount'].sum()

#Communications Category

applying Log transformation

In [ ]:
total_amount_cat_comm.head()

,Year-Month,Amount
75,2015-07,339296.73
76,2015-08,331352.60
77,2015-09,341914.45
78,2015-10,348125.98
79,2015-11,350533.37


In [ ]:
#changing the column names
total_amount_cat_comm.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_comm['ds'] = pd.to_datetime(total_amount_cat_comm['ds'])   #.dt.tz_convert(None)
total_amount_cat_comm.tail(n=3)

In [ ]:
#saving the original dataframe before making any transformaitons
total_amount_cat_comm_orig = total_amount_cat_comm.copy()
total_amount_cat_comm_orig.head()

In [ ]:
#Applying log transformation

total_amount_cat_comm['y'] = np.log(total_amount_cat_comm['y'])  #.diff(periods = 1) 

total_amount_cat_comm.dropna()
#NOW WE HAVE ds | y| y_transformed

total_amount_cat_comm.head()

In [ ]:
# total_amount_cat_comm = total_amount_cat_comm.reset_index() #removes the index from the data frame so we can assign 'ds', 'y' to the column names
#we are setting the difference column
# total_amount_cat_comm.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
# total_amount_cat_comm['ds'] = pd.to_datetime(total_amount_cat_comm['ds'])   #.dt.tz_convert(None)
# total_amount_cat_comm.tail(n=3)




In [ ]:
end_date = '2021-03-01'
mask1 = (total_amount_cat_comm['ds'] <= end_date)
mask2 = (total_amount_cat_comm['ds'] > end_date)

In [ ]:
train_df_comm = total_amount_cat_comm.loc[mask1]
test_df_comm = total_amount_cat_comm.loc[mask2]
print("train shape", train_df_comm.shape)
print("test shape", test_df_comm.shape)

In [ ]:
model_original_comm = Prophet()
model_original_comm.fit(train_df_comm);

In [ ]:
future_comm = model_original_comm.make_future_dataframe(periods = 6, freq = 'M')
future_comm.tail()

In [ ]:
forecast_comm = model_original_comm.predict(future_comm)
forecast_comm.tail(n=3)

In [ ]:
model_original_comm.plot(forecast_comm);

In [ ]:
model_original_comm.plot_components(forecast_comm);

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_comm =  model_original_comm.plot(forecast_comm)

fig_ch_comm = add_changepoints_to_plot(plot_changepoints_comm.gca(), model_original_comm, forecast_comm)


In [ ]:
# prediction_size = 6

# #split the data into train and test. For TRain, we will use everything except the last 6 months will be on the train data
# train_df_comm_diff1 = total_amount_cat_comm[:-prediction_size]
# train_df_comm_diff1.tail(n=3)

In [ ]:
# #we are using the last 6 months as the test dataset
# test_df_comm_diff1 = total_amount_cat_comm[-prediction_size:]
# test_df_comm_diff1.tail(n=3)

In [ ]:
# model_original_comm_diff1 = Prophet()
# model_original_comm_diff1.fit(train_df_comm_diff1);

In [ ]:
# future_comm_diff1 = model_original_comm_diff1.make_future_dataframe(periods = prediction_size)
# future_comm_diff1.tail(n=3)

In [ ]:
# forecast_comm_diff1 = model_original_comm_diff1.predict(future_comm_diff1)
# forecast_comm_diff1.tail(n=3)

In [ ]:
# model_original_comm_diff1.plot(forecast_comm_diff1);

In [ ]:
# model_original_comm_diff1.plot_components(forecast_comm_diff1);

In [ ]:
# #Looking at how many changepoints there are
# from fbprophet.plot import add_changepoints_to_plot

# plot_changepoints_comm_trans =  model_original_comm_diff1.plot(forecast_comm_diff1)

# fig_ch_comm_trans = add_changepoints_to_plot(plot_changepoints_comm_trans.gca(), model_original_comm_diff1, forecast_comm_diff1 )


In [ ]:
print(', '.join(forecast_comm.columns))

In [ ]:
X_tst_forecast_comm_log = model_original_comm.predict(test_df_comm)
X_tst_forecast_comm_log[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # Plot the forecast with the actuals(Keeping the reversed transformations)
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# ax.scatter(test_df_comm_diff1.ds, test_df_comm_diff1['y'], color='r')
# fig = model_original_comm_diff1.plot(X_tst_forecast_comm_diff1, ax=ax)

In [ ]:
#Plot the forecast vs Actuals(Keeping the transformations)

f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)


test_df_comm.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
X_tst_forecast_comm_log.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('April to September Forecast vs Actuals of Transformed Communication Trend line')
plt.show()

In [ ]:
#Calculating the accuracy before the transformations are reversed
MAPE_comm_trans_basic = mean_absolute_percentage_error(test_df_comm['y'], X_tst_forecast_comm_log['yhat'])
print('MAPE Score of Communications category before reversing predictions', MAPE_comm_trans_basic)

In [ ]:
# X_tst_forecast_comm_diff1_restored = X_tst_forecast_comm_log.copy().set_index('ds')
# X_tst_forecast_comm_diff1_restored.head()

In [ ]:
#https://github.com/tensorflow/tensorflow/issues/41326


Getting accuracy from original model after reversing both the predicted and test values

In [ ]:
from sklearn.metrics import mean_absolute_error  
# data = model_original_comm.predict(data)  

mean_absolute_error(np.exp(test_df_comm['y']), np.exp(X_tst_forecast_comm_log['yhat']))


#we’d probably want something more robust like cross-validation over sliding windows of the data set.

yhat_restored

In [ ]:
# #reversing the transformations
# X_tst_forecast_comm_diff1['yhat_restored_cumsum'] = X_tst_forecast_comm_diff1['yhat'].cumsum()
# X_tst_forecast_comm_diff1.head(5)

In [ ]:
# #Calculating the accuracy before the transformations are reversed
# MAPE_comm_trans_basic_reversed = mean_absolute_percentage_error(test_df_comm['y'], X_tst_forecast_comm_diff1['yhat_restored'])
# print('MAPE Score of Communications category before reversing predictions', MAPE_comm_trans_basic)

In [ ]:
# #Plotting the predictions vs actuals (after the transformations are reversed)
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_df_comm_diff1.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
# X_tst_forecast_comm_diff1.plot(kind='line', x='ds', y = 'yhat_restored', color = 'green', label = 'Forecast', ax=ax)
# plt.title('March to September 2020 Forecast vs Actuals')
# plt.show()


Performing Hyperparameter optimization on the transformed trend line

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               # 'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
#Steps to-do:
#NEed to change the start and end dates
#AFTER OBTAINING RESULTS, PERFORM REVERSE TRANSFORMATIONS ON Y_HAT, Y, Y_Lower(may need to delete null value rows)


In [ ]:
# # #creating the test and train split for our model
# train_df_comm_diff1_hyper = train_df_comm_diff1.copy().set_index('ds')
# train_df_comm_diff1_hyper.head()


In [ ]:
# test_df_comm_diff1_hyper = test_df_comm_diff1.copy().set_index('ds')
# test_df_comm_diff1_hyper.head()


In [ ]:
# strt='2019-04'
# end='2020-09'



# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                         #  weekly_seasonality=True,
#                         #  daily_seasonality = True,
#                         #  yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     # train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_df_comm_diff1_hyper)
#     train_forecast = train_model.make_future_dataframe(periods=5, freq='M',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]
#     Actual = total_amount_cat_comm[(total_amount_cat_comm['ds']>strt) & (total_amount_cat_comm['ds']<=end)]


# #accuracy of the MAPE score

#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)


In [ ]:
# #dataframe of combinantion of parameters with MAPE.

# parameters_trans_comm = model_parameters.sort_values(by=['MAPE'])
# parameters_trans_comm = parameters_trans_comm.reset_index(drop=True)
# parameters_trans_comm.head()

In [ ]:
# parameters_trans_comm['Parameters'][0]


In [ ]:
#Insert the values for the parameters that gave us the lowest MAPE.

# # Setup and train model with holidays
# final_model_trans_comm = Prophet( #holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_trans_comm.add_country_holidays(country_name='US')
# final_model_trans_comm.fit(train_df_comm_diff1_hyper)

In [ ]:
# future_comm_trans = final_model_trans_comm.make_future_dataframe(periods= 5, freq= 'M')
# forecast_comm_trans = final_model_trans_comm.predict(future_comm_trans)
# forecast_comm_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)



In [ ]:
# Plot the components of the model
# fig = final_model_trans_comm.plot_components(forecast_comm_trans)

In [ ]:
# # # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_trans_comm.plot(forecast_comm_trans,ax=ax)
# plt.show()

In [ ]:
# X_tst_final_comm_trans = final_model_trans_comm.predict(test_df_comm_diff1_hyper)
# X_tst_final_comm_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # #Plotting the actuals with the 

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_df_comm_diff1_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)



# #using the original trend line to plot
# X_tst_final_comm_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September Forecast vs Actuals')
# plt.show()

In [ ]:
# #Calculating MAPE score(need to account for Nan values to get an accurate prediciton score)
# MAPE = mean_absolute_percentage_error(X_tst_final_comm_trans['yhat'], abs(test_df_comm_diff1_hyper['yhat']))
# print('MAPE score of Communications category before reversing Transformation and Applying Hyperparameter Tuning: ', MAPE)

We need to reverse the transformation of 'yhat in X_tst_final so we can compare the reversed transformations Actual vs PRedicted

In [ ]:
# #CODE TO REVERSE THE 'yhat' in X_tst_final, to be able to compare it to the original model for communications category

# #reversing the transformations
# X_tst_final_comm_trans['yhat_restored'] = X_tst_final_comm_trans['yhat'].cumsum()
# X_tst_final_comm_trans.head(5)

In [ ]:
# # #Plotting the actuals with the 

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_df_comm_diff1_hyper.plot(kind='line', x='ds', y='y', color='red', label='Actual', ax=ax)



# #using the original trend line to plot
# X_tst_final_comm_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September Forecast vs Actuals')
# plt.show()

In [ ]:
# # #Calculating MAPE score(need to account for Nan values to get an accurate prediciton score)
# MAPE = mean_absolute_percentage_error(test_df_comm_hyper['y'], abs(test_df_comm_diff1_hyper['yhat']))
# print('MAPE score of Communications category after reversing Transformation and Applying Hyperparameter Tuning: ', MAPE)

#Retails Category

In [ ]:
# total_amount_cat_retails_original = total_amount_cat_retails.copy()
# total_amount_cat_retails_original.head()

#we want to work with the original dataframe with untransformed values first
# total_amount_cat_retails_orig = total_amount_cat_retails.copy()
total_amount_cat_retails.head()

In [ ]:
#
total_amount_cat_retails['Amount_log_rent'] = np.log(total_amount_cat_retails['y'])
total_amount_cat_retails['Amount_log_rent'].dropna()

#Let cross-check the differencing method scores with this log transformed data again.
total_amount_cat_retails['Amount_log_diff_rent'] = total_amount_cat_retails['Amount_log_rent'].diff(periods = 1) # First order differencing

total_amount_cat_retails['Amount_log_diff_rent'].dropna()# Data transformation may add na values


In [ ]:
total_amount_cat_retails = total_amount_cat_retails.drop(['y', 'Amount_log_rent'], axis = 1)
total_amount_cat_retails.head()

In [ ]:
total_amount_cat_retails = total_amount_cat_retails.rename(columns = {'Amount_log_diff_rent': 'y'})
total_amount_cat_retails.head()

Performing a basic prophet model on the transformed trend line

In [ ]:
#CHECK CODE TO SEE IF IT DOES PREPROCESSING CORRECTLY

In [ ]:
# total_amount_cat_retails = total_amount_cat_retails.reset_index()
# total_amount_cat_retails.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_retails['ds'] = pd.to_datetime(total_amount_cat_retails['ds'])   #.dt.tz_convert(None)
total_amount_cat_retails.tail(n=3)

In [ ]:
#splitting the data into train and test
end_date = '2021-03-01'
mask1 = (total_amount_cat_retails['ds'] <= end_date)
mask2 = (total_amount_cat_retails['ds'] > end_date)


In [ ]:
train_df_retail = total_amount_cat_retails.loc[mask1]
test_df_retail = total_amount_cat_retails.loc[mask2]
print("train shape",train_df_retail.shape)
print("test shape",test_df_retail.shape)

In [ ]:
model_original_retails = Prophet()
model_original_retails.fit(train_df_retail);


In [ ]:

future_retails = model_original_retails.make_future_dataframe(periods=6 , freq = 'M')
future_retails.tail(n=3)


In [ ]:
forecast_retails = model_original_retails.predict(future_retails)
forecast_retails.tail(n=3)

In [ ]:
model_original_retails.plot(forecast_retails);


In [ ]:
model_original_retails.plot_components(forecast_retails);


In [ ]:
# from fbprophet.plot import add_changepoints_to_plot

# plot_changepoints_retail =  model_original_retails.plot(forecast_retails)

# fig_ch_retail = add_changepoints_to_plot(plot_changepoints_retail.gca(), model_original_retails, forecast_retails )

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_retails_trans =  model_original_retails_logdiff.plot(forecast_logdiff_retails)

fig_ch_retail_trans = add_changepoints_to_plot(plot_changepoints_retails_trans.gca(), model_original_retails_logdiff, forecast_logdiff_retails )

In [ ]:
# prediction_size = 6
# train_df_retails_logdiff = total_amount_cat_retails[: -prediction_size]
# train_df_retails_logdiff.tail(n=3)

In [ ]:
# test_df_retails_logdiff = total_amount_cat_retails[ -prediction_size:]
# test_df_retails_logdiff.tail(n=3)

In [ ]:
# model_original_retails_logdiff = Prophet()
# model_original_retails_logdiff.fit(train_df_retails_logdiff);

In [ ]:
# future_log_diff_retials = model_original_retails_logdiff.make_future_dataframe(periods=prediction_size)
# future_log_diff_retials.tail(n=3)

In [ ]:

# forecast_logdiff_retails = model_original_retails_logdiff.predict(future_log_diff_retials)
# forecast_logdiff_retails.tail(n=3)


In [ ]:
# model_original_retails_logdiff.plot(forecast_logdiff_retails);


In [ ]:
# model_original_retails_logdiff.plot_components(forecast_logdiff_retails);


In [ ]:
print(','.join(forecast_logdiff_retails.columns))

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_retails_trans =  model_original_retails_logdiff.plot(forecast_logdiff_retails)

fig_ch_retail_trans = add_changepoints_to_plot(plot_changepoints_retails_trans.gca(), model_original_retails_logdiff, forecast_logdiff_retails )

Calculating the accuracy of basic prophet model on the transformed trend line

In [ ]:
X_tst_forecast_retails_transformed = model_original_retails_logdiff.predict(test_df_retails_logdiff)
X_tst_forecast_retails_transformed[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test_df_retails_logdiff.ds, test_df_retails_logdiff['y'], color = 'r')
fig = model_original_retails_logdiff.plot(X_tst_forecast_retails_transformed, ax = ax)



In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

test_df_retails_logdiff.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)
X_tst_forecast_retails_transformed.plot(kind = 'line', x = 'ds', y = 'yhat', color = 'green', label = 'Forecast', ax = ax)
plt.title('April to September Forecast vs Actuals on Transformed Retails Category Before Reversing')



In [ ]:
MAPE_trans_retails = mean_absolute_percentage_error(test_df_retails_logdiff['y'], X_tst_forecast_retails_transformed['yhat'])
print('Getting the basic prophet model on a transformed trend line without reversing: ', MAPE_trans_retails )

Reversing the Transformation to plot the reversed predictions from applying Basic prophet model on the transformed dataset. 

In [ ]:
# # X_tst_forecast_retails_transformed['yhat_restored']]

# last_revenue = X_tst_forecast_retails_transformed['yhat'].iloc[-1]

# X_tst_forecast_retails_transformed['yhat_restored'] = (np.exp(X_tst_forecast_retails_transformed['yhat'].cumsum())*last_revenue)

# X_tst_forecast_retails_transformed.head()

In [ ]:
# #WE NEED TO REVERSE THE TRANSFORMATIONS OF YHAT, yhat_lower, yhat_upper

# MAPE_trans_retails = mean_absolute_percentage_error(test_df_retails['y'], X_tst_forecast_retails_transformed['yhat_restored'])
# print('Getting the basic prophet model on a transformed trend line without reversing: ', MAPE_trans_retails )

In [ ]:
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)

# test_df_retails.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)
# X_tst_forecast_retails_transformed.plot(kind = 'line', x = 'ds', y = 'yhat_restored', color = 'green', label = 'Forecast', ax = ax)
# plt.title('April to September Forecast vs Actuals on Transformed Retails Category After Reversing Predicted Values')

Perform hyperparameter optimization on the transformed trend line

In [ ]:
# train_df_retails_logdiff_hyper = train_df_retails_logdiff.copy().set_index('ds')
# train_df_retails_logdiff_hyper.head()

In [ ]:
# train_df_retails_logdiff_hyper = train_df_retails_logdiff_hyper.reset_index()
# train_df_retails_logdiff_hyper.head()

In [ ]:
# test_df_retails_logdiff_hyper = test_df_retails_logdiff.copy().set_index('ds')
# test_df_retails_logdiff_hyper.head()

In [ ]:
# test_df_retails_logdiff_hyper = test_df_retails_logdiff_hyper.reset_index()
# test_df_retails_logdiff_hyper.head()

In [ ]:
# strt='2021-04'.  #Change the dates to include all the dates besides the last 3 months
# end='2021-09'


# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model_retails_trans =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model_retails_trans.add_country_holidays(country_name='US')
#     train_model_retails_trans.fit(train_df_retails_logdiff_hyper)
#     train_forecast = train_model_retails_trans.make_future_dataframe(periods=6, freq='M',include_history = False)
#     train_forecast = train_model_retails_trans.predict(train_forecast)
#     test= train_forecast[['ds','yhat']]
#     Actual = total_amount_cat_retails[(total_amount_cat_retails['ds']>strt) & (total_amount_cat_retails['ds']<=end)]



#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
# #THIS IS A DATAFRAME THAT CONTAINS THE PARAMETERS USED AS WELL AS THE MAPE score for each combination of parameters

# parameters_retail_trans = model_parameters.sort_values(by=['MAPE'])
# parameters_retail_trans = parameters_retail_trans.reset_index(drop=True)
# parameters_retail_trans.head()

In [ ]:
# parameters_retail_trans['Parameters'][0]


In [ ]:
# # # Setup and train model with holidays
# final_model_retail_trans = Prophet(#holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       # weekly_seasonality=True,
#                       # daily_seasonality = True,
#                       # yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_retail_trans.add_country_holidays(country_name='US')
# final_model_retail_trans.fit(train_df_retails_logdiff_hyper)


In [ ]:
# future_retail_trans = final_model_retail_trans.make_future_dataframe(periods= 6, freq='M')
# forecast_retail_transformed = final_model_retail_trans.predict(future_retail_trans)
# forecast_retail_transformed[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# Plot the components of the model

# fig = final_model.plot_components(forecast)

In [ ]:
# X_tst_final_retail_trans = final_model_retail_trans.predict(test_df_retails_logdiff_hyper)
# X_tst_final_retail_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# X_tst_final_retail_trans['yhat_restored'] = X_tst_final_retail_trans['']

Reverse the transformations on the results from  the hyperparameterized model

In [ ]:
#We need to reverse the transformation of the predicted values after performing Hyperparameter tuning on the Transformed trend line

Plot the accuracy of the 

In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)


# #plotting the original trend line 
# test_df_retails_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the reversed values after performing hyperparameter tuning on the transformed trend
# X_tst_final_rent_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan to June 2021 Forecast vs Actuals')
# plt.show()


In [ ]:
# #CALCULATING THE ACCURACY OF THE FINAL MODEL AFTER REVERSING THE PREDICTIONS
# MAPE = mean_absolute_percentage_error(test_df_retails_hyper['y'], abs(X_tst_final_rent_trans['yhat']))
# print('MAPE', MAPE)

#Rental Category

In [ ]:
#copying the original dataframe with columns 'ds' and 'y' for us to work with
total_amount_cat_rent_orig = total_amount_cat_rent.copy()
total_amount_cat_rent.head()

In [ ]:
total_amount_cat_rent['log_rent_amount'] = np.log(total_amount_cat_rent['y'])
total_amount_cat_rent['log_rent_amount'].dropna()# Data transformation may add na values


In [ ]:
#Let cross-check the differencing method scores with this log transformed data again.
total_amount_cat_rent['log_rent_diff_amount'] = total_amount_cat_rent['log_rent_amount'].diff(periods = 1) # First order differencing

# total_amount_cat_rent = 
total_amount_cat_rent['log_rent_diff_amount'].dropna(inplace = True)# Data transformation may add na values
# stationarity_test(total_amount_cat_rent['log_rent_diff_amount'])

total_amount_cat_rent.head()

In [ ]:
#we want to perform our basic prophet model on the Log difference order 1 transformation
total_amount_cat_rent = total_amount_cat_rent.drop(['y', 'log_rent_amount'], axis = 1)

In [ ]:
total_amount_cat_rent = total_amount_cat_rent.rename(columns = {'log_rent_diff_amount': 'y'})
total_amount_cat_rent.head()

In [ ]:
# total_amount_cat_rent = total_amount_cat_rent.reset_index()
# total_amount_cat_rent.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_rent['ds'] = pd.to_datetime(total_amount_cat_rent['ds'])   #.dt.tz_convert(None)
total_amount_cat_rent.tail(n=3)


Performing a basic prophet model on the transformed dataset

In [ ]:
end_date = '2021-03-01'
mask1 = (total_amount_cat_rent['ds'] <= end_date)
mask2 = (total_amount_cat_rent['ds'] > end_date)


In [ ]:
train_df_rent = total_amount_cat_rent.loc[mask1]
test_df_rent = total_amount_cat_rent.loc[mask2]
print("train shape",train_df_rent.shape)
print("test shape",test_df_rent.shape)

In [ ]:
model_original_rent = Prophet()
model_original_rent.fit(train_df_rent);

In [ ]:
future_rent = model_original_rent.make_future_dataframe(periods=6, freq = 'M')
future_rent.tail(n=3)


In [ ]:
forecast_rent = model_original_rent.predict(future_rent)
forecast_rent.tail(n=3)

In [ ]:
model_original_rent.plot(forecast_rent);


In [ ]:
model_original_rent.plot_components(forecast_rent);


In [ ]:
print(', '.join(forecast_rent.columns))


In [ ]:
X_test_forecast_rent = model_original_rent.predict(test_df_rent)
X_test_forecast_rent[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)


In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test_df_rent.ds, test_df_rent['y'], color = 'r')
fig = model_original_rent.plot(X_test_forecast_rent, ax = ax)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_rent =  model_original_rent.plot(forecast_rent)

fig_ch_rent= add_changepoints_to_plot(plot_changepoints_rent.gca(), model_original_rent, forecast_rent)


In [ ]:
# prediction_size = 6
# train_df_rent_trasnformed = total_amount_cat_rent[:-prediction_size]
# train_df_rent_trasnformed.tail(n=3)


In [ ]:
# test_df_rent_trasnformed = total_amount_cat_rent[-prediction_size:]
# test_df_rent_trasnformed.tail(n=3)

In [ ]:
# model_original_rent_transformed = Prophet()
# model_original_rent_transformed.fit(train_df_rent_trasnformed);

In [ ]:
# future_rent_transformed = model_original_rent_transformed.make_future_dataframe(periods=prediction_size)
# future_rent_transformed.tail(n=3)


In [ ]:
# forecast_rent_transformed = model_original_rent_transformed.predict(future_rent_transformed)
# forecast_rent_transformed.tail(n=3)


In [ ]:
# model_original_rent_transformed.plot(forecast_rent_transformed);


In [ ]:
# model_original_rent_transformed.plot_components(forecast_rent_transformed);


In [ ]:
# print(' , '.join(forecast_rent_transformed.columns))

In [ ]:
# X_test_forecast_rent_trans = model_original_rent_transformed.predict(test_df_rent_trasnformed)
# X_test_forecast_rent_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)

#plotting the original trend line of 
test_df_rent.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)


X_test_forecast_rent.plot(kind = 'line', x = 'ds', y = 'yhat', color = 'green', label = 'Forecast', ax = ax)

plt.title('April to September 2021 Forecast vs Actuals of Transformed rental category')
plt.show()


In [ ]:
MAPE = mean_absolute_percentage_error(test_df_rent_trasnformed['y'], X_test_forecast_rent_trans['yhat'])
print('MAPE Score of basic prophet model on Transformed Rental Category trend line: ',MAPE )

REverse the transformation of the predicted values(We need to reverser lof differenceing 

In [ ]:
# last_revenue = X_test_forecast_rent_trans['yhat'].iloc[-1]

# X_test_forecast_rent_trans['yhat_restored']=(np.exp(X_test_forecast_rent_trans['yhat'].cumsum())*last_revenue)



# X_test_forecast_rent_trans.head()

In [ ]:
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)

# #plotting the original trend line of Rentals category
# test_df_rent_hyper.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)


# #Plotting the 
# X_test_forecast_rent_trans.plot(kind = 'line', x = 'ds', y = 'yhat_restored', color = 'green', label = 'Forecast', ax = ax)

# plt.title('April to September 2021 Forecast vs Actuals of Transformed rental category after reversing predictions')
# plt.show()


In [ ]:
# MAPE_trans_rent_reversed = mean_absolute_percentage_error(test_df_rent_trasnformed['y'], X_test_forecast_rent_trans['yhat'])
# print('MAPE Score of basic prophet model on Transformed Rental Category trend line after reversing predictions: ', MAPE_trans_rent_reversed )

Perform Hyperpramter tuning on the transformed dataset

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# train_df_rent_trasnformed_hyper = train_df_rent_trasnformed.copy().set_index('ds')
# train_df_rent_trasnformed_hyper.head()


In [ ]:
# train_df_rent_trasnformed_hyper = train_df_rent_trasnformed_hyper.reset_index()
# train_df_rent_trasnformed_hyper.head()

In [ ]:
# test_df_rent_trasnformed_hyper =test_df_rent_trasnformed.copy().set_index('ds')
# test_df_rent_trasnformed_hyper.head()


In [ ]:
# test_df_rent_trasnformed_hyper =test_df_rent_trasnformed.reset_index()
# test_df_rent_trasnformed_hyper.head()

In [ ]:
#CHANGE THE START AND END DATE TO INCLUDE BEGINNING OF THE DATA AND EVERYTHING EXCEPT THE LAST 6 MONTHS
# strt='2021-04'
# end='2021-09'

# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_df_rent_trasnformed_hyper)
#     train_forecast = train_model.make_future_dataframe(periods=57, freq='D',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test = train_forecast[['ds','yhat']] #INCLUDE y_hat, y_lower
#     Actual = total_amount_cat_rent[(total_amount_cat_rent['ds']>strt) & (total_amount_cat_rent['ds']<=end)]



#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
#sorting the rows in the parameters dataframe by MAPE score

# parameters_rent = model_parameters.sort_values(by=['MAPE'])
# parameters_rent = parameters_rent.reset_index(drop=True)
# parameters_rent.head()

In [ ]:
#getting the parameters for the model with the lowest MAPE score w
# parameters_rent['Parameters'][0]


In [ ]:
# # Setup and train model with holidays
# final_model_rent_trans = Prophet(#holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_rent_trans.add_country_holidays(country_name='US')
# final_model_rent_trans.fit(train_df_rent_trasnformed_hyper)

In [ ]:
# future_rent_trans = final_model_rent_trans.make_future_dataframe(periods= 5, freq='M')
# forecast_rent_trans = final_model_rent_trans.predict(future_rent_trans)
# forecast_rent_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# Plot the components of the model
# fig = final_model_rent_trans.plot_components(forecast_rent_trans)

In [ ]:
# # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_rent_trans.plot(forecast_rent_trans,ax=ax)
# plt.show()

In [ ]:
# X_tst_final_rent_trans = forecast_rent_trans.predict(test_df_rent_trasnformed_hyper)
# X_tst_final_rent_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
#Comparison dataframe. ADDING THE ORIGINAL DATAFRAME WITH THE YValues, and y_hat, y_lower, y_upper

# cmp_df = make_comparison_dataframe(total_amount_cat_rent,  X_tst_final['y'])
# cmp_df.tail(n=3)

In [ ]:
#SHOWING THE VISUALS BETWEEN PREDICTED VS ACTUALS

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_df_rent_trasnformed_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
# X_tst_final_rent_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan through June 2021 Forecast vs Actuals keeping predicted values transformed')
# plt.show()

In [ ]:
# for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
#     print(err_name, err_value)

Reversing the transformation of the 'yhat' in X_tst_final_rent_trans

In [ ]:
#CODE TO REVERSE THE TRANSFORMATION

In [ ]:
#PLOT AFTER REVERSING THE PREDICTED VALUES

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_df_rent_trasnformed_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
# X_tst_final_rent_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan through June 2021 Forecast vs Actuals after Revers')
# plt.show()

In [ ]:
#we want to compare the accuracy to the original dataframe with predicted values that are not transformed
# MAPE = mean_absolute_percentage_error(test_df_rent_hyper['y'],abs(X_tst_final_rent_trans['yhat']))
# print('MAPE', MAPE)

In [ ]:
# show_forecast(cmp_df, prediction_size, 100, 'New posts on Medium')


#Restaurant and bars

We will also use log for our stationary transformation for Restuarant and Bars

In [ ]:
# total_amount_cat_rest_n_bars = total_amount_cat_rest_n_bars.reset_index()
total_amount_cat_rest_n_bars.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_rest_n_bars['ds'] = pd.to_datetime(total_amount_cat_rest_n_bars['ds'])   #.dt.tz_convert(None)
total_amount_cat_rest_n_bars.tail(n=3)

In [ ]:
# Starting with the original dataframe with columns 'ds', 'y' and untransformed values
total_amount_cat_rest_n_bars_orig = total_amount_cat_rest_n_bars.copy()
total_amount_cat_rest_n_bars_orig.head()

In [ ]:
end_date = '2021-03-01'
mask1 = (total_amount_cat_rest_n_bars['ds'] <= end_date)
mask2 = (total_amount_cat_rest_n_bars['ds'] > end_date)


In [ ]:
train_df_restnbars = total_amount_cat_rest_n_bars.loc[mask1]
test_df_restnbars = total_amount_cat_rest_n_bars.loc[mask2]
print("train shape",train_df_restnbars.shape)
print("test shape",test_df_restnbars.shape)


In [ ]:
model_original_restnbars = Prophet()
model_original_restnbars.fit(train_df_restnbars);

In [ ]:
future_restnbars = model_original_restnbars.make_future_dataframe(periods=6, freq = 'M')
future_restnbars.tail(n=3)

In [ ]:
forecast_restnbars = model_original_restnbars.predict(future_restnbars)
forecast_restnbars.tail(n=3)

In [ ]:
model_original_restnbars.plot(forecast_restnbars);


In [ ]:
model_original_restnbars.plot_components(forecast_restnbars);


In [ ]:
print(', '.join(forecast_restnbars.columns))


In [ ]:
#Plot to see the change points
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_restnbars =  model_original_restnbars.plot(forecast_restnbars)

fig_ch_restnbars = add_changepoints_to_plot(plot_changepoints_restnbars.gca(), model_original_restnbars, forecast_restnbars)


In [ ]:
X_test_forecast_restnbars =   model_original_restnbars.predict(test_df_restnbars)
X_test_forecast_restnbars[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)



In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test_df_restnbars.ds, test_df_restnbars['y'], color = 'r')
fig =   model_original_restnbars.plot(X_test_forecast_restnbars, ax = ax)

In [ ]:
f, ax = plt.subplots(figsize = (14, 5))
f.set_figheight(5)
f.set_figwidth(5)

test_df_restnbars.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)
X_test_forecast_restnbars.plot(kind = 'line', x = 'ds', y = 'yhat', color = 'green', label = 'Forecast', ax = ax)

plt.title('April tp September 2021 Forecast vs ACtuals for Resturant and Bars Category applying simple prophet model')
plt.show()


In [ ]:
#Calculating MAPE score before reversing any transformations
MAPE_restnbars_basic = mean_absolute_percentage_error(test_df_restnbars['y'], X_test_forecast_restnbars['yhat'])
print('MAPE score of basic Prophet model on the Restaurant and Bars category : ', MAPE_restnbars_basic)

Calculate accuracy after reversing the log transformations

In [ ]:
#Calculating MAPE score before reversing any transformations
MAPE_restnbars_basic_reversed = mean_absolute_percentage_error(np.exp(test_df_restnbars['y']), np.exp(X_test_forecast_restnbars['yhat']))
print('MAPE score of basic Prophet model on the Restaurant and Bars category after Reversing: ', MAPE_restnbars_basic_reversed)

Reversing the predicted values

In [ ]:
# # CODE TO Reverse the transformation 'yhat'  on the X_tst_forecast_restnbars_diff1 dataframe 
# X_tst_forecast_restnbars_diff1['yhat_restored'] = X_tst_forecast_restnbars_diff1['yhat'].cumsum()

# X_tst_forecast_restnbars_diff1.head()



In [ ]:
# MAPE_restnbars_trans_reversed = mean_absolute_percentage_error(test_df_restnbars_hyper['y'], X_tst_forecast_restnbars_diff1['yhat_restored'])
# print('MAPE score of transformed trend line for Restaurant and BArs Category after reversing: ', round(MAPE_restnbars_trans_reversed, 4))

PErforming Hyperparameter optimization on the transformed trend line

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# train_df_diff_restnbars.copy().set_index('ds')
# train_df_diff_restnbars.head()


In [ ]:
# train_df_diff_restnbars = train_df_diff_restnbars.reset_index()
# train_df_diff_restnbars.tail()

In [ ]:
# test_df_diff_restnbars.copy().set_index('ds')
# test_df_diff_restnbars.head()

In [ ]:
# test_df_diff_restnbars = test_df_diff_restnbars.reset_index()
# test_df_diff_restnbars.tail()

In [ ]:
# strt='2021-04'
# end='2021-09' #THE END DATE IS THE 6 months from the last date


# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_df_diff_restnbars)
#     train_forecast = train_model.make_future_dataframe(periods=6, freq='M',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]
#     Actual = total_amount_cat_rest_n_bars[(total_amount_cat_rest_n_bars['ds']>strt) & (total_amount_cat_rest_n_bars['ds']<=end)]


#     #Calculating the accuracy
#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
# #Listing the model combinations and the MAPE

# parameters_restnbars_trans = model_parameters.sort_values(by=['MAPE'])
# parameters_restnbars_trans = parameters_restnbars_trans.reset_index(drop=True)
# parameters_restnbars_trans.head()

In [ ]:

#Getting the parameters of the model which has the lowest MAPE score.
# parameters_restnbars_trans['Parameters'][0]


In [ ]:
# Setup and train model with holidays
# final_model_restnbars_trans = Prophet(#holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_restnbars_trans.add_country_holidays(country_name='US')
# final_model_restnbars_trans.fit(train_df_diff_restnbars)

In [ ]:
# future_restnbars_trans = final_model_restnbars_trans.make_future_dataframe(periods=6, freq='M')
# forecast_restnbars_trans = final_model_restnbars_trans.predict(future_restnbars_trans)
# forecast_restnbars_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # Plot the components of the model
# fig = final_model_restnbars_trans.plot_components(forecast_restnbars_trans)

In [ ]:
# # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_restnbars_trans.plot(forecast_restnbars_trans, ax=ax)
# plt.show()

In [ ]:
# X_tst_final_restnbars_trans = final_model_restnbars_trans.predict(test_df_diff_restnbars)
# X_tst_final_restnbars_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plotting the Actuals
# test_df_diff_restnbars.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the forecasted
# X_tst_final_restnbars_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

Reversing the transformation of 'yhat' in X_tst_final_restnbars_trans

In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plotting the Actuals
# test_df_diff_restnbars.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the forecasted values of the transformed trend line after reversing the predicted values
# X_tst_final_restnbars_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
#Getting the MAPE score of the 

# MAPE = mean_absolute_percentage_error(test_df_diff_restnbars['y'], abs(X_tst_final_restnbars_trans['yhat']))
# print('MAPE', MAPE)

#Utilities

In [ ]:
total_amount_cat_util_orig = total_amount_cat_util.copy()
total_amount_cat_util_orig.head()

In [ ]:
total_amount_cat_util['Revenue_Diff1'] = total_amount_cat_util['y'].diff(periods = 1)
total_amount_cat_util = total_amount_cat_util.dropna()

total_amount_cat_util.head()

In [ ]:
total_amount_cat_util = total_amount_cat_util.drop('y', axis = 1)
total_amount_cat_util.head()

In [ ]:
# total_amount_cat_util = total_amount_cat_util.reset_index()
total_amount_cat_util = total_amount_cat_util.rename(columns = {'Revenue_Diff1': 'y'})

# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_util['ds'] = pd.to_datetime(total_amount_cat_util['ds'])   #.dt.tz_convert(None)
total_amount_cat_util.tail(n=3)


In [ ]:
end_date = '2021-03-01'
mask1 = (total_amount_cat_util['ds'] <= end_date)
mask2 = (total_amount_cat_util['ds'] > end_date)


In [ ]:
train_df_util = total_amount_cat_util.loc[mask1]
test_df_util = total_amount_cat_util.loc[mask2]
print("train shape",train_df_util.shape)
print("test shape",test_df_util.shape)

In [ ]:
model_original_util = Prophet()
model_original_util.fit(train_df_util);

In [ ]:
future_util = model_original_util.make_future_dataframe(periods= 6, freq = 'M')
future_util.tail(n=3)

In [ ]:
forecast_util = model_original_util.predict(future_util)
forecast_util.tail(n=3)


In [ ]:
model_original_util.plot_components(forecast_util);


In [ ]:
#Plot to see the change points
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_util =  model_original_util.plot(forecast_util)

fig_ch_util = add_changepoints_to_plot(plot_changepoints_util.gca(), model_original_util, forecast_util)


In [ ]:
print(', '.join(forecast_util.columns))

In [ ]:
X_test_forecast_util = model_original_util.predict(test_df_util)

X_test_forecast_util[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)


In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test_df_util.ds, test_df_util['y'], color = 'r')
fig =  model_original_util.plot(X_test_forecast_util, ax = ax)


In [ ]:
f, ax = plt.subplots(1)
figsize = (14, 5)
f.set_figheight(5)
f.set_figwidth(5)

test_df_util.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)


X_test_forecast_util.plot(kind = 'line', x = 'ds', y = 'yhat', color = 'green', label = 'Forecast', ax = ax)
plt.title('April to September 2021 Forecast vs ACtuals')
plt.show()


In [ ]:
MAPE_basic_util = mean_absolute_percentage_error(total_amount_cat_util['y'], abs(forecast_util['yhat']))
print('MAPE Score of basic prophet model on Utilities category: ', round(MAPE_basic_util, 5))

In [ ]:
# prediction_size = 6
# train_df_util_transformed = total_amount_cat_util[:-prediction_size]
# train_df_util_transformed.tail(n=3)

In [ ]:
# test_df_util_transformed = total_amount_cat_util[-prediction_size:]
# test_df_util_transformed.tail(n=3)

In [ ]:
# model_util_transformed = Prophet()
# model_util_transformed.fit(train_df_util_transformed);

In [ ]:
# future_util_transformed = model_util_transformed.make_future_dataframe(periods = prediction_size)
# future_util_transformed.tail(n=3)


In [ ]:
# forecast_util_transformed = model_util_transformed.predict(future_util_transformed)
# forecast_util_transformed.tail(n=3)


In [ ]:
# model_util_transformed.plot(forecast_util_transformed);


In [ ]:
# model_util_transformed.plot_components(forecast_util_transformed);


In [ ]:
# print(', '.join(forecast_util_transformed.columns))


In [ ]:
# x_tst_forecast_util_diff1 = model_util_transformed.predict(test_df_util_transformed)
# x_tst_forecast_util_diff1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# #PRedicted values vs actual values 

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plottint the original
# test_df_util_transformed.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the forecasted values
# x_tst_forecast_util_diff1.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
# MAPE = mean_absolute_percentage_error(test_df_util_transformed['y'], x_tst_forecast_util_diff1['yhat'])
# print('MAPE score of basic prophet model on the transformed trend line for utilitiies without reversing: ', MAPE)

Reverse the yhat column in x_tst_forecast_util_diff1

In [ ]:
# #reversing the differencing order 1 and storing it in another vairable
# x_tst_forecast_util_diff1['yhat_restored'] = x_tst_forecast_util_diff1['yhat'].cumsum()
# x_tst_forecast_util_diff1.head()

In [ ]:
# #PRedicted values vs actual values 

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plottint the original
# test_df_util.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the forecasted values
# x_tst_forecast_util_diff1.plot(kind='line',x='ds',y='yhat_restored', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
# MAPE = mean_absolute_percentage_error(test_df_util_transformed['y'], x_tst_forecast_util_diff1['yhat'])
# print('MAPE score of basic prophet model on the transformed trend line for utilitiies After reversing predicted: ', MAPE)

Perform Hyperparameterization on the transformed model

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# train_df_util_transformed_hyper = train_df_util_transformed.copy().set_index('ds') 
# train_df_util_transformed_hyper.tail(n=3)

In [ ]:
# train_df_util_transformed_hyper = train_df_util_transformed_hyper.reset_index()
# train_df_util_transformed_hyper.tail()

In [ ]:
# test_df_util_transformed_hyper = train_df_util_transformed.copy().set_index('ds') 
# test_df_util_transformed_hyper.tail(n=3)

In [ ]:
# test_df_util_transformed_hyper = test_df_util_transformed_hyper.reset_index()
# test_df_util_transformed_hyper.tail()

In [ ]:
# strt='2021-04'
# end='2021-09'
# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_df_util_transformed_hyper)
#     train_forecast = train_model.make_future_dataframe(periods=6, freq='M',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]
#     Actual = total_amount_cat_util[(total_amount_cat_util['ds']>strt) & (total_amount_cat_util['ds']<=end)]


#     #Calculating the accuracy of the MAPE scores
#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
#making a dataframe for parameters usedand the MAPE

# parameters_util = model_parameters.sort_values(by=['MAPE'])
# parameters_util = parameters_util.reset_index(drop=True)
# parameters_util.head()

In [ ]:
#looking at the MAPE score
# parameters_util['Parameters'][0]


In [ ]:
# Setup and train model with holidays
# final_model_util_trans = Prophet(#holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_util_trans.add_country_holidays(country_name='US')
# final_model_util_trans.fit(train_df_util_transformed_hyper)

In [ ]:
# future_util_trans = final_model_util_trans.make_future_dataframe(periods= 5, freq='M')
# forecast_util_trans = final_model_util_trans.predict(future_util_trans)
# forecast_util_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# Plot the components of the model
# fig = final_model_util_trans.plot_components(forecast_util_trans)

In [ ]:
# Plot the forecast

# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_util_trans.plot(forecast_util_trans,ax=ax)
# plt.show()

In [ ]:
# X_tst_final_util_trans = final_model_util_trans.predict(test_df_util_transformed_hyper)
# X_tst_final_util_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
#plotting the Predicted vs aCtual before reversing transformation
#PRedicted values vs actual values 

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plottint the original amount vlaues that are not 
# test_df_util_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the forecasted values
# X_tst_final_util_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals with reversed predicted values')
# plt.show()

Reverse the 'y_hat', 'y_lower', 'y_upper' in X_tst_final_util_trans

In [ ]:
#CODE TO REVERSE THOSE column values 


In [ ]:
# #Plotting the predicted vs actual

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #
# test_df_util_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #
# X_tst_final_util_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan & Feb 2020 Forecast vs Actuals')
# plt.show()

#Contracting

In [ ]:
# total_amount_cat_cont_original = total_amount_cat_cont.copy()
# total_amount_cat_cont_original.head()

total_amount_cat_cont_orig = total_amount_cat_cont.copy()
total_amount_cat_cont.head()

In [ ]:
#creating the transformed variable
total_amount_cat_cont['Revenue_Diff1'] = total_amount_cat_cont['y'].diff(periods = 1)
total_amount_cat_cont = total_amount_cat_cont.dropna()
total_amount_cat_cont.head()

In [ ]:
total_amount_cat_cont = total_amount_cat_cont.drop('y', axis = 1)
total_amount_cat_cont.head()

In [ ]:
total_amount_cat_cont = total_amount_cat_cont.rename(columns = {'Revenue_Diff1': 'y'})
total_amount_cat_cont.head()

In [ ]:
# total_amount_cat_cont = total_amount_cat_cont.reset_index()
# total_amount_cat_cont.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
total_amount_cat_cont['ds'] = pd.to_datetime(total_amount_cat_cont['ds'])   #.dt.tz_convert(None)
total_amount_cat_cont.tail(n=3)


Performing a Basic prophet model on the transformed trend line

In [ ]:
end_date = '2021-03-01'
mask1 = (total_amount_cat_cont['ds'] <= end_date)
mask2 = (total_amount_cat_cont['ds'] > end_date)

In [ ]:
train_df_cont = total_amount_cat_cont.loc[mask1]
test_df_cont = total_amount_cat_cont.loc[mask2]
print("train shape",train_df_cont.shape)
print("test shape",test_df_cont.shape)

In [ ]:
model_original_cont = Prophet()
model_original_cont.fit(train_df_cont);


In [ ]:
future_cont = model_original_cont.make_future_dataframe(periods=6, freq = 'M')
future_cont.tail(n=3)


In [ ]:
forecast_cont = model_original_cont.predict(future_cont)
forecast_cont.tail(n=3)

In [ ]:
model_original_cont.plot(forecast_cont);

In [ ]:
model_original_cont.plot_components(forecast_cont);


In [ ]:
print(', '.join(forecast_cont.columns))


In [ ]:
#Plot to see the change points
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_cont =  model_original_cont.plot(forecast_cont)

fig_ch_cont = add_changepoints_to_plot(plot_changepoints_cont.gca(), model_original_cont, forecast_cont)



Actual vs Predicted Comparsion

In [ ]:
X_tst_forecast_cont = model_original_cont.predict(test_df_cont)
X_tst_forecast_cont[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(X_tst_forecast_cont.ds, X_tst_forecast_cont['yhat'], color = 'r')
fig = model_original_cont.plot(X_tst_forecast_cont, ax = ax)

In [ ]:
f, ax = plt.subplots(figsize = (14, 5))
f.set_figheight(5)
f.set_figwidth(5)

test_df_cont.plot(kind = 'line', x = 'ds', y = 'y', color = 'red', label = 'Test', ax = ax)

X_tst_forecast_cont.plot(kind = 'line', x = 'ds', y = 'yhat', color = 'green', label = 'Forecast', ax = ax)
plt.title('February tp September 2021 Forecast vs ACtuals')
plt.show()

In [ ]:
MAPE_basic_cont = mean_absolute_percentage_error(total_amount_cat_cont['y'], abs(forecast_cont['yhat']))
print('MAPE score of basic Prophet model on Contrcting category: ', round(MAPE_basic_cont, 4))


Reverse the transformation

In [ ]:
# x_tst_forecast_cont_diff1['yhat_restored'] = x_tst_forecast_cont_diff1['yhat'].cumsum()
# x_tst_forecast_cont_diff1.head()

In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)


# #Forecast of 
# test_df_cont.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Forecast of predicted values
# x_tst_forecast_cont_diff1.plot(kind = 'line', x = 'ds', y = 'yhat_restored', color = 'green', label = 'Forecast', ax = ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
# MAPE = mean_absolute_percentage_error(test_df_cont['y'], x_tst_forecast_cont_diff1['yhat'])
# print('MAPE score of the basic prophet model on the transformed trend line after reversing the predicted values: ', MAPE)


Perforing Hyperparameter optimization on the transfomred trend line

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# train_df_cont_hyper = train_df_cont_trans.copy().set_index('ds')
# train_df_cont_hyper.tail()

In [ ]:
# train_df_cont_hyper = train_df_cont_hyper.reset_index()
# train_df_cont_hyper.tail()

In [ ]:
# test_df_cont_hyper = test_df_cont_trans.copy().set_index('ds')
# test_df_cont_hyper.tail()

In [ ]:
# test_df_cont_hyper = test_df_cont_hyper.reset_index()
# test_df_cont_hyper.tail()

In [ ]:
# strt='2021-04'
# end='2021-09'  #Change the start and end date


# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     # random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_df_cont_hyper)
#     train_forecast = train_model.make_future_dataframe(periods=6, freq='M',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]
#     Actual = total_amount_cat_cont[(total_amount_cat_cont['ds']>strt) & (total_amount_cat_cont['ds']<=end)]


#       #Calculating the accuracy of all the combinations using MAPE score
#     MAPE = mean_absolute_percentage_error(Actual['y'], abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
# #creating a parameter dataframe with combniation of parameters and the MAPE score. 

# parameters_cont_trans = model_parameters.sort_values(by=['MAPE'])
# parameters_cont_trans = parameters_cont_trans.reset_index(drop=True)
# parameters_cont_trans.head()

In [ ]:
# #best parameters which the model has the lowest MAPE score

# parameters_cont_trans['Parameters'][0]


In [ ]:
# # Setup and train model with holidays
# final_model_cont_trans = Prophet(#holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_cont_trans.add_country_holidays(country_name='US')
# final_model_cont_trans.fit(train_df_cont_hyper)

In [ ]:
# future_cont_trans = final_model_cont_trans.make_future_dataframe(periods=6, freq='M')
# forecast_cont_trans = final_model_cont_trans.predict(future_cont_trans)
# forecast_cont_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # Plot the components of the model
# fig = final_model_cont_trans.plot_components(forecast_cont_trans)

In [ ]:
# # # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_cont_trans.plot(forecast_cont_trans,ax=ax)
# plt.show()


In [ ]:
# X_tst_final_cont_trans = final_model_cont_trans.predict(test_df_cont_hyper)
# X_tst_final_cont_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #Plotting the Actual trend line
# test_df_cont_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

# #Plotting the predicted trend line
# X_tst_final_cont_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan to June 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
# #Calculating the MAPE score
# MAPE = mean_absolute_percentage_error(test_df_cont_hyper['y'],abs(X_tst_final['yhat']))
# print('MAPE score keeping the transformed dataset and predicted values', MAPE)

Reversing the transformation of the predicted values

In [ ]:
#REVERSE THE TRANSFOMRATIONS OF yhat, yhat_lower, yhat_upper on 


#Categories that we Aggregated

In [ ]:
# agg_cat_dfs_orig = agg_cat_dfs.copy()
# agg_cat_dfs_orig.head()
agg_cat_dfs = tlt_pandA_publ_amuesement_df_monthly.copy()
agg_cat_dfs.head()

In [ ]:
agg_cat_dfs['Revenue_Diff1'] = agg_cat_dfs['y'].diff(periods = 1)
agg_cat_dfs = agg_cat_dfs.dropna()
agg_cat_dfs.head()

In [ ]:
agg_cat_dfs = agg_cat_dfs.drop('y', axis = 1)
agg_cat_dfs.head()

In [ ]:
agg_cat_dfs = agg_cat_dfs.rename(columns = {'Revenue_Diff1': 'y'})
agg_cat_dfs.head()
# agg_cat_dfs = agg_cat_dfs.reset_index()


In [ ]:
agg_cat_dfs['ds'] = pd.to_datetime(agg_cat_dfs['ds'])
agg_cat_dfs.head()

PErforming Basic prophet model on the transformed trend line

In [ ]:
end_date = '2021-03-01'
mask1 = (agg_cat_dfs['ds'] <= end_date)
mask2 = (agg_cat_dfs['ds'] > end_date)

In [ ]:
train_aggdf = agg_cat_dfs.loc[mask1]
test_aggdf = agg_cat_dfs.loc[mask2]
print("train shape", train_aggdf.shape)
print("test shape", test_aggdf.shape)

In [ ]:
model_original_agg = Prophet()
model_original_agg.fit(train_aggdf);

In [ ]:
future_agg = model_original_agg.make_future_dataframe(periods= 6, freq = 'M')
future_agg.tail(n=3)

In [ ]:
forecast_agg = model_original_agg.predict(future_agg)
forecast_agg.tail(n=3)

In [ ]:
model_original_agg.plot(forecast_agg);


In [ ]:
model_original_agg.plot_components(forecast_agg);


In [ ]:
print(', '.join(forecast_agg.columns))


In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model_original_agg.plot(forecast_agg,ax=ax)
plt.show()

In [ ]:
#Plot to see the change points
from fbprophet.plot import add_changepoints_to_plot

plot_changepoints_aggdfs =  model_original_agg.plot(forecast_agg)

fig_ch_aggdf = add_changepoints_to_plot(plot_changepoints_aggdfs.gca(), model_original_agg, forecast_agg)


In [ ]:
#modeling the 
X_tst_forecast_aggregated_dfs = model_original_agg.predict(test_aggdf)
X_tst_forecast_aggregated_dfs[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(test_aggdf.ds, test_aggdf['y'], color='r')
fig = model_original_agg.plot(X_tst_forecast_aggregated_dfs, ax=ax)

In [ ]:
f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)
test_aggdf.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)
X_tst_forecast_aggregated_dfs.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('February-Sep 2021 Forecast vs Actuals')
plt.show()

In [ ]:
#MAPE score
MAPE_basic_aggdfs = mean_absolute_percentage_error(test_aggdf['y'], abs(X_tst_forecast_aggregated_dfs['yhat']))
print('MAPE score of Basic Prophet model on aggregated dataframes ', round(MAPE_basic_aggdfs, 5) )

In [ ]:
# prediction_size = 6
# train_aggdfs_trans = agg_cat_dfs[:-prediction_size]
# train_aggdfs_trans.tail(n=3)

In [ ]:
# train_aggdfs_trans.shape

In [ ]:
# test_aggdfs_test = agg_cat_dfs[-prediction_size:]
# test_aggdfs_test.tail(n=3)

In [ ]:
# test_aggdfs_test.shape

In [ ]:
# model_aggdfs_diff1 = Prophet()
# model_aggdfs_diff1.fit(train_aggdfs_trans);

In [ ]:
# future_aggdfs_diff1 = model_aggdfs_diff1.make_future_dataframe(periods=prediction_size, freq = 'M')
# future_aggdfs_diff1.tail(n=3)

In [ ]:
# forecast_aggdfs_diff1 = model_aggdfs_diff1.predict(future_aggdfs_diff1)
# forecast_aggdfs_diff1.tail(n=3)

In [ ]:
# model_aggdfs_diff1.plot(forecast_aggdfs_diff1);


In [ ]:
# model_aggdfs_diff1.plot_components(forecast_aggdfs_diff1);


In [ ]:
# print(', '.join(forecast_aggdfs_diff1.columns))


In [ ]:
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = model_aggdfs_diff1.plot(forecast_aggdfs_diff1,ax=ax)
# plt.show()

In [ ]:
# X_tst_forecast_aggdfs_trans = model_aggdfs_diff1.predict(test_aggdfs_test)
# X_tst_forecast_aggdfs_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # Plot the forecast with the actuals
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# ax.scatter(test_aggdfs_test.ds, test_aggdfs_test['y'], color='r')
# fig = model_aggdfs_diff1.plot(X_tst_forecast_aggdfs_trans, ax=ax)

In [ ]:
#Actuals vs predicted

f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)

#plotting the actuals (keeping it transformed)
test_aggdfs_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)


#Plotting the Predictions(Keeping the transformed varaible)
X_tst_forecast_aggdfs_trans.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('February-Sep 2021 Forecast vs Actuals')
plt.show()

In [ ]:
mape = mean_absolute_percentage_error(test_aggdfs_test['y'], X_tst_forecast_aggdfs_trans['yhat'])
print("MAPE score of basic prophet model on transformed trend line of aggregated datasets Before Reversing: ", round(mape,4))

Reversing the transformations on the forecast 'yhat'

In [ ]:
# #Reversing the differencing order 1 
# X_tst_forecast_aggdfs_trans['yhat_restored'] = X_tst_forecast_aggdfs_trans['yhat'].cumsum()
# X_tst_forecast_aggdfs_trans.head()

In [ ]:
# #Actuals vs predicted

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)

# #plotting the actuals (keeping it transformed)
# test_aggdfs_test.plot(kind='line',x='ds', y='y', color='red', label='Test', ax=ax)


# #Plotting the Predictions(Keeping the transformed varaible)
# X_tst_forecast_aggdfs_trans.plot(kind='line',x='ds',y='yhat_restored', color='green',label='Forecast', ax=ax)
# plt.title('February-Sep 2021 Forecast vs Actuals')
# plt.show()

PErforming hyperparameter tuning on the transformed trend lnie

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# train_aggdf_hyper_trans = train_aggdfs_trans.copy().set_index('ds')
# train_aggdf_hyper_trans.head()

In [ ]:
# train_aggdf_hyper_trans = train_aggdf_hyper_trans.reset_index()
# train_aggdf_hyper_trans.tail()

In [ ]:
# test_aggdfs_hyper_trans = test_aggdfs_trans.copy().set_index('ds')
# test_aggdfs_hyper_trans.head()

In [ ]:
# test_aggdfs_hyper_trans = test_aggdfs_hyper_trans.reset_index()
# test_aggdfs_hyper_trans.tail()

In [ ]:
# strt='2021-04'
# end='2021-09'

# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality=True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                         #  holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(train_aggdf_hyper_trans)
#     train_forecast = train_model.make_future_dataframe(periods=6, freq='M',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]
#     Actual = agg_cat_dfs[(agg_cat_dfs['ds']>strt) & (agg_cat_dfs['ds']<=end)]

# # Getting the accuracy of the MAPE score

#     MAPE = mean_absolute_percentage_error(Actual['y'], abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
# #dataframe of parameters and the MAPE

# parameters_aggdfs_trans = model_parameters.sort_values(by=['MAPE'])
# parameters_aggdfs_trans = parameters_aggdfs_trans.reset_index(drop=True)
# parameters_aggdfs_trans.head()

In [ ]:
# #Combinaition of parameters that give the lowest MAPE score

# parameters_aggdfs_trans['Parameters'][0]


In [ ]:
# # Setup and train model with holidays
# final_model_aggdfs_trans = Prophet( #holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model_aggdfs_trans.add_country_holidays(country_name='US')
# final_model_aggdfs_trans.fit(train_aggdf_hyper_trans)

In [ ]:
# future_aggdfs_trans = final_model_aggdfs_trans.make_future_dataframe(periods=6, freq='M')
# forecast_aggdfs_trans = final_model_aggdfs_trans.predict(future_aggdfs_trans)
# forecast_aggdfs_trans[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# Plot the components of the transformed model
# fig = final_model_aggdfs_trans.plot_components(forecast_aggdfs_trans)

In [ ]:
# # # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model_aggdfs_trans.plot(forecast_aggdfs_trans,ax=ax)
# plt.show()

In [ ]:
# X_tst_final= final_model_aggdfs_trans.predict(test_aggdfs_hyper_trans)
# X_tst_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
#Plotting the predicted vs actuals before reversing anything

# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# test_aggdfs_hyper_trans.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
# X_tst_final.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('April to September 2021 Forecast vs Actuals')
# plt.show()

Reverse the 'yhat', 'yhat_lower', 'yhat_upper'

In [ ]:
# #MAPE score of hyperparameterization on the transformed  trend line

# MAPE = mean_absolute_percentage_error(test_agg_df['y'],abs(X_tst_final['yhat']))
# print('MAPE', MAPE)

#Original Model

In [ ]:
# total_amount_cat_cont_original = total_amount_cat_cont.copy()
# total_amount_cat_cont_original.head()
mesa_az = mesa_az_orig.copy()
mesa_az.head()


In [ ]:
#creating the transformed variable
mesa_az['Revenue_Diff1'] = mesa_az['y'].diff(periods = 1)
mesa_az = mesa_az.dropna()
mesa_az.head()

In [ ]:
mesa_az = mesa_az.drop('y', axis = 1)
mesa_az.head()


In [ ]:
mesa_az = mesa_az.rename(columns = {'Revenue_Diff1': 'y'})
mesa_az.head()

In [ ]:
# total_amount_cat_cont = total_amount_cat_cont.reset_index()
# total_amount_cat_cont.columns = ['ds', 'y']
# converting timezones (issue https://github.com/facebook/prophet/issues/831)
mesa_az['ds'] = pd.to_datetime(mesa_az['ds'])   #.dt.tz_convert(None)
mesa_az.tail(n=3)


In [ ]:
prediction_size = 6

In [ ]:
train_df_mesa_trans = mesa_az[:-prediction_size]
train_df_mesa_trans.tail()

In [ ]:
train_df_mesa_trans.shape

In [ ]:
test_df_mesa_trans = mesa_az[-prediction_size:]
test_df_mesa_trans.tail()


In [ ]:
test_df_mesa_trans.shape

In [ ]:
model_mesa_transformed = Prophet()
model_mesa_transformed.fit(train_df_mesa_trans);

In [ ]:
future_mesa_transformed = model_mesa_transformed.make_future_dataframe(periods = prediction_size, freq = 'M')
future_mesa_transformed.tail(n=3)

In [ ]:
forecast_mesa_transformed = model_mesa_transformed.predict(future_mesa_transformed)
forecast_mesa_transformed.tail(n=3)

In [ ]:
model_mesa_transformed.plot(forecast_mesa_transformed);


In [ ]:
model_mesa_transformed.plot_components(forecast_mesa_transformed);


In [ ]:
print(', '.join(forecast_mesa_transformed.columns))


In [ ]:
x_tst_forecast_mesa_diff1 = model_mesa_transformed.predict(test_df_mesa_trans)
x_tst_forecast_mesa_diff1[['ds', 'yhat', 'yhat_upper', 'yhat_lower']].tail(7)


In [ ]:
#Plotting the predicted vs actuals before reversing both of them
f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)

#Forecast of 
test_df_mesa_trans.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

#Forecast of predicted values
x_tst_forecast_mesa_diff1.plot(kind='line',x='ds', y='yhat', color='green',label='Forecast', ax=ax)
plt.title('April to September 2021 Forecast vs Actuals')
plt.show()

In [ ]:
MAPE = mean_absolute_percentage_error(test_df_mesa_trans['y'], x_tst_forecast_mesa_diff1['yhat'])
print('MAPE score of the basic prophet model on the transformed trend line: ', MAPE)


Reversing the trasformations of the predicted values

In [ ]:
x_tst_forecast_mesa_diff1['yhat_restored'] = x_tst_forecast_mesa_diff1['yhat'].cumsum()
x_tst_forecast_mesa_diff1.head()

In [ ]:
#Plotting the predicted vs actuals before reversing both of them
f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)

#Forecast of 
test_df_meza_hyper.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)

#Forecast of predicted values
x_tst_forecast_mesa_diff1.plot(kind='line',x='ds', y='yhat_restored', color='green',label='Forecast', ax=ax)
plt.title('April to September 2021 Forecast vs Actuals')
plt.show()

In [ ]:
MAPE = mean_absolute_percentage_error(test_df_meza_hyper['y'], x_tst_forecast_mesa_diff1['yhat_restored'])
print('MAPE score of the basic prophet model on the transformed trend line: ', MAPE)


Performing Hyperparameter optimization on the most staionary trend line

In [ ]:
# train_df_meza_hyper_trans =  train_df_mesa_trans.copy().set_index('ds')
# train_df_meza_hyper_trans.head()

In [ ]:
# train_df_meza_hyper_trans = train_df_meza_hyper_trans.reset_index()
# train_df_meza_hyper_trans.head()

In [ ]:
# test_df_meza_hyper_trans = test_df_mesa_trans.copy().set_index('ds')
# test_df_meza_hyper_trans.head()

In [ ]:
# test_df_meza_hyper_trans = test_df_meza_hyper_trans.reset_index()
# test_df_meza_hyper_trans.head()

In [ ]:
# from sklearn.model_selection import ParameterGrid
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200]}
# grid = ParameterGrid(params_grid)
# cnt = 0
# for p in grid:
#     cnt = cnt+1

# print('Total Possible Models',cnt)

In [ ]:
# strt='2021-04'.  #Changing the start and end date
# end='2021-09'
# model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
# for p in grid:
#     test = pd.DataFrame()
#     print(p)
#     random.seed(0)
#     train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
#                          holidays_prior_scale = p['holidays_prior_scale'],
#                          n_changepoints = p['n_changepoints'],
#                          seasonality_mode = p['seasonality_mode'],
#                          weekly_seasonality = True,
#                          daily_seasonality = True,
#                          yearly_seasonality = True,
#                          holidays=holiday, 
#                          interval_width=0.95)
#     train_model.add_country_holidays(country_name='US')
#     train_model.fit(X_tr)
#     train_forecast = train_model.make_future_dataframe(periods=57, freq='D',include_history = False)
#     train_forecast = train_model.predict(train_forecast)
#     test=train_forecast[['ds','yhat']]

#       #Calculating the accuracy of each combination of parameters and thier MAPE score 
      
#     Actual = meza_total_monthly[(meza_total_monthly['ds']>strt) & (meza_total_monthly['ds']<=end)]
#     MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
#     print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)
#     model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)

In [ ]:
#dataframe of combination of parameters used and thier MAPE score

# parameters = model_parameters.sort_values(by=['MAPE'])
# parameters = parameters.reset_index(drop=True)
# parameters.head()

In [ ]:
# Best Parameters on which the model has the least MAPE is:

# parameters['Parameters'][0]

In [ ]:
# # Setup and train model with holidays
# final_model = Prophet(holidays=holiday,
#                       changepoint_prior_scale= 0.5,
#                       holidays_prior_scale = 0.1,
#                       n_changepoints = 200,
#                       seasonality_mode = 'multiplicative',
#                       weekly_seasonality=True,
#                       daily_seasonality = True,
#                       yearly_seasonality = True,
#                       interval_width=0.95)
# final_model.add_country_holidays(country_name='US')
# final_model.fit(X_tr)

In [ ]:
# future = final_model.make_future_dataframe(periods = 6, freq='M')
# forecast = final_model.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
# # Plot the components of the model
# fig = final_model.plot_components(forecast)


In [ ]:

# # Plot the forecast
# f, ax = plt.subplots(1)
# f.set_figheight(5)
# f.set_figwidth(15)
# fig = final_model.plot(forecast,ax=ax)
# plt.show()

In [ ]:


# X_tst_final= final_model.predict(X_tst)
# X_tst_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
#REverse the transformations of the predicted values 

#perform the inverse difference operatinos
#Perform exp() function to inverse the log 


In [ ]:
# f, ax = plt.subplots(figsize=(14,5))
# f.set_figheight(5)
# f.set_figwidth(15)
# X_tst.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
# X_tst_final.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
# plt.title('Jan to June 2021 Forecast vs Actuals')
# plt.show()

In [ ]:
# MAPE = mean_absolute_percentage_error(X_tst['y'],abs(X_tst_final['yhat']))
# print('MAPE', MAPE)

#Misc.

Differencing order is difficult to reverse. From doing research, we need to know the original values so tha we can reverse the difference.

It would however, be very easy to reverse transformations of log transformations. We will try these out first.

Categories that greatly benefit from log: Communications, Restaurant and Bars, Aggregated